In [71]:
%load_ext autoreload
%autoreload 2
import pandas as pd
import re
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.externals import joblib

from sklearn.pipeline import Pipeline, FeatureUnion

from pipeline.text_cleaner import TextCleaner
from pipeline.stemmer import Stemmer
from pipeline.dataframe_vectorizer import DataframeVectorizer

from sklearn import metrics
import pickle

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [72]:
df = pd.DataFrame()
df = pd.read_csv("data/imdb_small.csv", sep="|")

In [74]:
X_train, X_test, y_train, y_test = \
    train_test_split(df[['text']], df['label'], test_size=0.2, random_state=42, stratify=df['label'])

In [75]:
DataPipeline = Pipeline(steps=[
    ('clean_words', TextCleaner(key='text')),
    ('stem', Stemmer()),
    ('vectorize', DataframeVectorizer(vectorizer=CountVectorizer()))])

In [76]:
LearningPipeline = Pipeline(steps=[('logistic', LogisticRegression())])

In [77]:
X_after_data = DataPipeline.fit_transform(X_train)

In [81]:
with open('dumps/data_pipeline.bin', 'wb') as f:
    pickle.dump(DataPipeline, f)

In [85]:
LearningPipeline.fit(X_after_data, y_train)

Pipeline(steps=[('logistic', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [86]:
with open('dumps/learn_pipeline.bin', 'wb') as f:
    pickle.dump(LearningPipeline, f)

In [87]:
X_test_after_data = DataPipeline.transform(X_test)

In [88]:
predicted = LearningPipeline.predict(X_test_after_data)

In [89]:
print(metrics.classification_report(y_test, predicted))

             precision    recall  f1-score   support

          0       0.89      0.88      0.89      5000
          1       0.88      0.90      0.89      5000

avg / total       0.89      0.89      0.89     10000

